## Startup Process
```mermaid
flowchart LR
    C[Serial.begin] --> D[strip.begin]
    D --> E[updateLength/setBrightness]
    E --> H[init stripData n oldStripData]
    H --> I[initializeBLE_fn]
```


> **Description:**
>
> The startup process initializes the LED strip controller by setting up serial communication, configuring the LED strip parameters (length, brightness, color order), and allocating memory for the strip data structures. It then instantiates the Bluetooth process by calling `initializeBLE`, which prepares the device for wireless communication. Only after these steps are complete does the main loop begin, allowing the system to respond to incoming messages and update the LED strip in real time. This ensures that all hardware and communication interfaces are ready before any data processing or animation occurs.

## Bluetooth Process
```mermaid
flowchart TD
    J[onWrite event]
    J --> K[parseAndUpdateData_fn]
    K --> L[myData.updated = true]
```


> **Description:**
>
> The `parseAndUpdateData` function is called when a Bluetooth message is received. It takes a JSON string and parses it into the `myData` structure, updating fields such as brightness, colors, mode, and other parameters. If parsing is successful, it sets `myData.updated = true`, signaling that new data has arrived and the LED strip should be refreshed with the new settings. This flag ensures that the main loop processes the new configuration and applies changes to the hardware.

## Main Loop Process
```mermaid
flowchart TD
    A[loop_fn] --> B[handleStrip_fn]
    B -- 1.mydata.updated --> E[cloneData_fn, setBrightness, length, type] 
    E --> H[strip.clear then show]
    H --> J[transitionValue = 100]
    B -- 2.update.stripData --> K[callModeFunction_fn]
    B -- 3.display.stripData --> L{transitionValue < 2?}
    L -- Yes --> M[show_fn]
    L -- No --> N[blendAndShow_fn]
    M --> O[strip.setPixelColor]
    O --> P[strip.show]
    N --> Q[blendColors_fn]
    Q --> R[strip.setPixelColor]
    R --> S[strip.show]
```

> **Description:**
>
> The `handleStrip` function is the core routine that manages updates to the LED strip. When `myData.updated` is true, it refreshes the strip settings, copies the previous data for transitions, and prepares the strip for new effects. It then calls `callModeFunction`, which applies the selected lighting mode based on the current configuration. The strip is updated either directly or through blending, depending on the value of `transitionValue`. If a transition is in progress, colors are smoothly blended between the old and new states. The `updated` parameter acts as a trigger, ensuring that changes are only processed when new data arrives, making the system responsive to Bluetooth or other input events.

# Lighting Basics

- Static: Passive, no animation.
- Dynamic: Active, animates over time.
- Modifier: Alters or combines effects; can be static or dynamic, and can be layered.

# myData Attributes

| Attribute      | Type      | Description                                               |
|---------------|-----------|-----------------------------------------------------------|
| brightness    | int       | LED strip brightness (0-100)                              |
| lightMode     | char[32]  | Current lighting mode (e.g., "static", "shift")          |
| colorOne      | uint32_t  | Primary color (RGB, 0xRRGGBB)                             |
| colorTwo      | uint32_t  | Secondary color (RGB, 0xRRGGBB)                           |
| colorThree    | uint32_t  | Tertiary color (RGB, 0xRRGGBB)                            |
| ledCount      | int       | Number of LEDs in the strip                               |
| maxCurrent    | int       | Maximum allowed current (mA)                              |
| colorOrder    | uint16_t  | Color order for the strip (e.g., NEO_GRB + NEO_KHZ800)    |
| pixelPin      | int       | GPIO pin connected to the LED strip                       |
| pixelCount    | int       | Number of pixels (usually same as ledCount)               |
| speed         | int       | Animation speed (0-100)                                   |
| intensity     | int       | Effect intensity (0-100)                                  |
| direction     | int       | Animation direction (0=forward, 1=reverse, 2=bounce)      |
| count         | int       | Count parameter for effects (e.g., number of segments)     |
| updated       | bool      | True if new data has arrived and should be processed       |

# Effects and Modes

### **File Organization**
- `lighting_modes_baselayer.cpp` - Base layer implementations
- `lighting_modes_overlay.cpp` - Overlay mode implementations  
- `lighting_modes_modifier.cpp` - Both static and dynamic modifier implementations
- `lighting_modes.cpp` - Main dispatcher and forward declarations

## EFFECTS

| Function Name         | Parameters                                          | Description                                                                                    |
|-----------------------|-----------------------------------------------------|------------------------------------------------------------------------------------------------|
| effect_static         | C                                                   | Sets all LEDs to the specified color                                                           |
| effect_blink          | BG, blinkInterval                                   | Blinks between source and background data at the given interval                                |
| effect_fade           | intensity                                           | Fades the LED data by the given intensity                                                      |
| effect_range          | startPixel, endPixel, fadeIntensity                 | Applies fade to a range of pixels from startPixel to endPixel                                  |
| effect_shift          | direction                                           | Shifts the LED data in the specified direction                                                 |
| effect_swipe          | direction, C, overridePixelIndex                    | Lights pixels one at a time along a strip, optionally overriding a pixel index                 |
| effect_sweep          | direction, C, dragLength, count, overridePixelIndex | Moves groups of LEDs across the strip with drag and count, optionally overriding a pixel index |
| effect_breath         | intensity                                           | Fades between colors to create a breathing effect                                              |


## Lighting Modes Reference Tables

### Base Layer Modes

| Mode Function | Parameters/Dependencies | Effect Function | Description |
|---------------|------------------------|-----------------|-------------|
| `mode_static` | colorOne | effect_static | Single solid color fill |
| `mode_static_tri` | colorOne, colorTwo, colorThree | effect_static | Three-color sections |
| `mode_perlin_move` | speed, pixelCount, fadeRate | -- | Perlin Noise movement |
| `mode_sunrise` | palette, time, width | -- | Simulates gradual sunrise/sunset |
| `mode_aurora` | palette, speed, intensity | -- | Aurora Borealis simulation |
| `mode_pacifica` | palette, speed, angle | -- | Gentle ocean waves |
| `mode_stream` | -- | effect_shift | Bands of random hues |
| `mode_palette` | -- | -- | Color palette cycling |
| `mode_plasma` | -- | -- | Plasma lamp effect |

### Overlay Modes

| Mode Function | Parameters/Dependencies | Effect Function | Description |
|---------------|------------------------|-----------------|-------------|
| `mode_swipe` | palette, colorOne, direction | effect_swipe | Lights pixels one at a time |
| `mode_swipe_random` | direction, randomColor | effect_swipe | Random color wipe |
| `mode_sweep` | direction | effect_sweep | Moves groups of LEDs across the strip |
| `mode_sweep_dual` | speed, trail | effect_sweep | Sinelon from both directions |
| `mode_theater` | speed, gapSize | effect_sweep | Theater effect with rainbow colors |
| `mode_candle` | speed, intensity | -- | Flicker resembling a candle flame |
| `mode_fireworks` | palette, speed, intensity | -- | A pixel shoots forward then explodes |
| `mode_juggle` | speed, trail | -- | Colored dots running, leaving trails |
| `mode_bouncing_balls` | gravity, ballCount | -- | Bouncing ball effect |
| `mode_tetrix` | speed, width, oneColor | -- | Falling blocks stack |
| `mode_meteor` | speed, trailLength | -- | Meteor trail effect |
| `mode_colorloop` | speed | Wheel | Rainbow color cycling |

### Modifier Modes

| Mode Function | Parameters/Dependencies | Effect Function | Description |
|---------------|------------------------|-----------------|-------------|
| `mode_percent` | palette, colorOne, colorTwo, intensity | effect_range | Lights up a percentage of segment |
| `mode_percent_tri` | intensity, colorOne, colorTwo, colorThree | effect_range | Tri-color percentage |
| `mode_blink` | palette, loopInterval | effect_blink | Blinks between primary color and off |
| `mode_blink_toggle` | speed, colorOne, colorTwo | effect_blink | Toggle between colors |
| `mode_blink_random` | speed, randomColor | effect_blink | Random color flashing |
| `mode_breath` | speed, inherits colors | effect_blink | Fades between primary and secondary color |
| `mode_heartbeat` | speed, intensity | effect_fade | Pulsing rhythm like a heartbeat |
| `mode_twinkles` | palette, fade, spawn, intensity | effect_fade | Colors fade in/out randomly |
| `mode_shift` | speed, intensity, inherits colors | effect_shift | Shifts pixels in a direction |
| `mode_washing_machine` | speed, intensity | effect_shift | Spins, slows, reverses directions |

overlays vs modifiers are dubious.
overlays vs  displays are dubious

need to break it into (twinkle_off-> intensity modifier) and (twinkle_on -> color overlay)

## Mission Objective: User-Composed Light Shows 

Composition System
Users would create Sequences where each segment is a Mode Stack:

Sequence: "My Light Show"
├── Segment 1 (0-10s): 
│   ├── Base: mode_pacifica (ocean waves)
│   ├── Overlay: mode_meteor (shooting stars)
│   └── Modifier: mode_blink (subtle pulse)
├── Segment 2 (10-20s):
│   ├── Base: mode_plasma 
│   ├── Overlay: mode_meteor (shooting stars)
│   └── Overlay: mode_fireworks
├── Segment 3 (20-30s):
│   ├── Base: mode_plasma 
│   ├── Base: mode_stream 
│   └── Modifier: mode_blink_toggle (subtle pulse) 



ovewritable default colors for select modes 
same mode is consistent between segments

two color systems:

Direct Colors: colorOne, colorTwo, colorThree from struct_message - user picks exact RGB values
Palettes: Used in effects like mode_plasma with SEGMENT.color_from_palette() - predefined color schemes


Each effect could have envelope parameters for Delay + ADSR:

Attack: How quickly effect fades in
Decay: Initial fade after peak
Sustain: Steady-state level
Release: Fade out time

UI Considerations
The UI would need several panels:

Timeline Editor: Horizontal timeline showing sequence segments
Layer Stack: For each segment, show base/overlay/modifier layers
Effect Library: Drag-and-drop available modes
Parameter Panel: Adjust colors, speed, intensity, ADSR for selected effect
Color/Palette Selector: Unified color system (need to resolve the dual system)

Technical Challenges to Solve
Color System Unification: Decide whether to use direct colors OR palettes, or create a hybrid system
Memory Management: Multiple overlapping effects will need careful memory allocation
Timing System: Need precise timing for ADSR and sequence transitions
Data Structure: Extend beyond current struct_message to handle complex compositions

Recommended Next Steps
Standardize Color System: Make all effects work with both user colors AND palettes
Create Composition Data Structure: Design how to store/transmit complex light shows
Build Simple Stacking: Start with 2-layer combinations (base + overlay)
Add ADSR to One Effect: Prototype the envelope system
Design Timeline UI: Mockup the user interface flow

Improvements: 
- Shift amount, merge Blink and Breathe vs Fade, swipe merge w sweep using persist(overridePixelIndex)
- effect_blend - Selects every n pixels to blend sideways
- Perlin / random selected indices for bleaching and such
- Segmenting
- Bezier/ sigmoid for all speed intensity fade etc.

> Juggle, Bouncing Balls, Meteor, Tetrix can use a common underlying function that animates moving objects (dots, balls, blocks) w (position, speed, trail, and color). 
> Plasma and Perlin Move can share a noise-based rendering function with different parameters. 
> Stream and Palette are more pattern-based, but could use a shared function for color cycling or zone-based coloring.
